In [1]:
import os, sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
# --- 
from techminer2 import *

directory = "regtech"

import_scopus_files(directory)
# import_references()
# annual_scientific_production()
# main_information(directory)

Note: to be able to use all crisp methods, you need to install some additional packages:  {'karateclub', 'leidenalg', 'infomap', 'wurlitzer', 'graph_tool'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'karateclub', 'ASLPAw'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'leidenalg'}
--INFO-- Creating `words.txt` from author/index keywords, and abstract/title words
--INFO-- Applying `words.txt` thesaurus to author/index keywords and abstract/title words
--INFO-- The regtech/processed/countries.txt thesaurus file was created
--INFO-- The regtech/processed/countries.txt thesaurus file was applied to affiliations in all databases
--INFO-- The regtech/processed/institutions.txt thesaurus file was created
--INFO-- The regtech/processed/institutions.txt thesaurus file was applied to affiliations in all databases


In [ ]:
import pandas as pd

directory = "regtech"

documents = pd.read_csv(f"{directory}/processed/_documents.csv")
references = pd.read_csv(f"{directory}/processed/_references.csv")

documents.article


In [ ]:
cited_references = documents.global_references.copy()
cited_references = cited_references.str.lower()
cited_references = cited_references.str.split(";")
cited_references = cited_references.explode()
cited_references = cited_references.str.strip()
cited_references = cited_references.dropna()
cited_references = cited_references.drop_duplicates()
cited_references = cited_references.reset_index(drop=True)

thesaurus = {t: None for t in cited_references.tolist()}

references['found'] = False


#
# Busqueda por DOI
#
for doi, article in zip(references.doi, references.article):
    for key in thesaurus.keys():
        if not pd.isna(doi) and doi in key:
            thesaurus[key] = article
            references.found[references.doi == doi] = True

#
# Reduce la base de búsqueda
#
references = references[references.found == False]

#
# Busqueda por año, autor y tttulo
#
for article, year, authors, title in zip(
    references.article, 
    references.year,
    references.authors,
    references.title,
):
    year = str(year)
    author = authors.split()[0].lower()
    title = title.lower().replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("-", " ").replace("'", "")
    
    for key in thesaurus.keys():
        text = key
        text = text.lower()
        text = text.replace(".", "")
        text = text.replace(",", "")
        text = text.replace(":", "")
        text = text.replace(";", "")
        text = text.replace("-", " ")
        text = text.replace("'", "")

        if (
            author in text and 
            str(year) in text and 
            title[:29] in text
        ):
            thesaurus[key] = article
            references.found[references.article == article] = True
        elif (
            author in text and 
            str(year) in text and 
            title[-29:] in text
        ):
            thesaurus[key] = article
            references.found[references.article == article] = True


#
# Reduce la base de búsqueda
#
references = references[references.found == False]

#
# Busqueda por titulo
#
for article, title in zip(
    references.article, 
    references.title,
):
    
    
    title = title.lower().replace(".", "").replace(",", "").replace(":", "")
    
    for key in thesaurus.keys():
        text = key
        text = text.lower()
        text = text.replace(".", "")
        text = text.replace(",", "")
        text = text.replace(":", "")

        if (            
            title in text
        ):
            thesaurus[key] = article
            references.found[references.article == article] = True




with open("report.txt", "w") as f:
    for key, value in thesaurus.items():
        if value is None:
            f.write(f"{key}\n")
    
    f.write("\n-----------------------\n\n")

    for article, found, title in zip(
        references.article, 
        references.found,
        references.title,
        
    ):
        if found is False:
            f.write(f"{article} --- {title}\n")

for key in thesaurus.keys():
    if thesaurus[key] is not None:
        print(thesaurus[key])

In [3]:
import pandas as pd

directory = "regtech"
pd.read_csv(f"{directory}/processed/_documents.csv").local_references

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
     ... 
89    NaN
90    NaN
91    NaN
92    NaN
93    NaN
Name: local_references, Length: 94, dtype: object

In [14]:
def _create__local_citations__column_in_documents_database(directory):

    # sys.stdout.write(
    #     "--INFO-- Creating `local_citations` column in references database\n"
    # )

    # counts the number of citations for each local reference
    documents_path = os.path.join(directory, "processed", "_documents.csv")
    documents = pd.read_csv(documents_path)
    local_references = documents.local_references.copy()
    local_references = local_references.dropna()
    local_references = local_references.str.split(";")
    local_references = local_references.explode()
    local_references = local_references.str.strip()
    local_references = local_references.value_counts()
    values_dict = local_references.to_dict()

    # assigns the number of citations to each reference in references database
    documents["local_citations"] = documents.article
    documents["local_citations"] = documents["local_citations"].map(values_dict)
    documents["local_citations"] = documents["local_citations"].fillna(0)

    # saves the new column in the references database
    documents.to_csv(documents_path, index=False)

directory = "regtech"
_create__local_citations__column_in_documents_database(directory)
